In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Input, Dense, MaxPooling2D, Conv2D, Flatten
from keras.optimizers import Adam
from keras import regularizers
from keras import optimizers
from keras import initializers
from keras.preprocessing import image

import keras.backend as K

from sklearn.preprocessing import OneHotEncoder


In [ ]:
trainfile = pd.read_csv("train.csv")

In [ ]:
trainfile.head()

In [ ]:
trainfile.shape

In [ ]:
xtrain = trainfile.iloc[:,1:].values.astype('float32')
ytrain = trainfile.iloc[:,0].values.astype('int')

xtrain = xtrain/255


xtrain = xtrain.reshape(xtrain.shape[0], 28, 28, 1)

In [ ]:
xtrain.shape

In [ ]:
enc = OneHotEncoder(n_values=10)

enc.fit(ytrain)

labels = enc.transform(ytrain).toarray()

#ytrain[:10]

ylabels = labels.reshape(ytrain.shape[0], 10)

#blabla = labEnc.transform(ytrain).toarray()

#blabla.shape

In [ ]:
plt.imshow(xtrain[6,:,:,0])

In [ ]:
def model(Xshape):
    
    mod = Sequential()
    mod.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='glorot_normal', input_shape=Xshape))
    mod.add(MaxPooling2D(pool_size=(2, 2)))
    
    mod.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='glorot_normal'))
    mod.add(MaxPooling2D(pool_size=(2, 2)))

    mod.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='glorot_normal'))
    mod.add(MaxPooling2D(pool_size=(2, 2)))

    mod.add(Flatten())
    mod.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0.01)))
    mod.add(Dense(128, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0.01)))
    mod.add(Dense(64, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0.01)))
    mod.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))
    
    return mod

In [ ]:
def model2(Xshape):
    
    mod = Sequential()
    
    mod.add(Conv2D(16, (5, 5), activation='relu', kernel_initializer='glorot_normal', input_shape=Xshape))
    mod.add(MaxPooling2D(pool_size=(2, 2)))

    mod.add(Conv2D(32, (5, 5), activation='relu', kernel_initializer='glorot_normal'))
    mod.add(MaxPooling2D(pool_size=(2, 2)))

    mod.add(Flatten())
    mod.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0.005)))
    mod.add(Dense(128, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0.005)))
    mod.add(Dense(64, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0.005)))
    mod.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))
    
    return mod

In [ ]:
mymodel = model2(xtrain.shape[1:])
opt = optimizers.Adam(lr=0.01, decay=0.0001)


mymodel.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
# First train with small mini batch size and default learning rate
mymodel.fit(xtrain, ylabels, epochs = 20, batch_size=128)
# Then train with smaller learning rate and larger batch size
K.set_value(mymodel.optimizer.lr, 0.001)
mymodel.fit(xtrain, ylabels, epochs = 10, batch_size=512)
K.set_value(mymodel.optimizer.lr, 0.0001)
mymodel.fit(xtrain, ylabels, epochs = 10, batch_size=1024)


In [ ]:
testfile = pd.read_csv("test.csv")
xtest = testfile.values.astype('float32')

xtest = xtest/255

xtest = xtest.reshape(xtest.shape[0], 28, 28, 1)

In [ ]:
predictions = mymodel.predict_classes(xtest)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("preds.csv", index=False, header=True)